In [13]:
# ingest_all_cves.ipynb  –  pull every CVE for every CPE in the whitelist
import os, time, requests, pandas as pd
from pathlib import Path

# ── config ─────────────────────────────────────────────────────────────
api_url      = "https://services.nvd.nist.gov/rest/json/cves/2.0"
api_key      = os.getenv("NVD_api_key") or "ea5501a5-24fe-4720-80e3-2abed401d92f"
whitelist    = Path("../data/cpe_whitelist.csv")
rate_secs    = 1.0
per_page     = 2000
progress_every = 25                # how often to print a status line
# ───────────────────────────────────────────────────────────────────────

def fetch_cves_for_cpe(cpe_uri: str) -> list[dict]:
    parts = cpe_uri.split(":")
    if len(parts) < 6:
        return []
    cpe_query = ":".join(parts[:6]) if parts[5] == "*" else cpe_uri

    all_items, start = [], 0
    headers = {"apiKey": api_key}
    
    while True:
        params = {
            "cpeName":        cpe_query,
            "resultsPerPage": per_page,
            "startIndex":     start,
        }
        r = requests.get(api_url, headers=headers, params=params, timeout=30)
        if r.status_code != 200:
            print(f"⚠️ {cpe_query[:70]} → {r.status_code}")
            break

        data   = r.json()
        items  = data.get("vulnerabilities", [])
        all_items.extend(items)

        start += per_page
        if start >= data.get("totalResults", 0) or not items:
            break
        time.sleep(rate_secs)
    return all_items

def flatten(v: dict, cpe_uri: str) -> dict:
    cve      = v["cve"]
    metrics  = cve.get("metrics", {})
    cvss31   = metrics.get("cvssMetricV31", [{}])[0].get("cvssData", {})
    cvss30   = metrics.get("cvssMetricV30", [{}])[0].get("cvssData", {})
    cvss     = cvss31 or cvss30
    descr    = next((d["value"] for d in cve.get("descriptions", []) if d["lang"] == "en"), "")
    cwes     = [
        d["value"] for w in cve.get("weaknesses", [])
        for d in w.get("description", []) if d.get("lang") == "en"
    ]

    refs     = "|".join(r["url"] for r in cve.get("references", [])[:10])
    tags = ", ".join(tag for r in cve.get("references", [])[:10] for tag in r.get("tags", []))
    
    
    
    return {
        "cveID":            cve["id"],
        "cpeName":          cpe_uri,
        "published":         cve.get("published"),
        "last_modified":     cve.get("lastModified"),
        "vectorString": 
        "cwes":              ";".join(cwes) if cwes else None,
        "description":       descr[:1000],
        "references":        refs,
        "tags":              tags,
        "full_json":         v,
    }

# ── 1. load whitelist ─────────────────────────────────────────────────
cpe_list = (
    pd.read_csv(whitelist, dtype=str)["cpeName"]
      .dropna()
      .unique()
)
print(f"📋  {len(cpe_list):,} unique CPEs to query")

# ── 2. query API ───────────────────────────────────────────────────────
rows = []
for idx, cpe in enumerate(cpe_list, start=1):
    if idx % progress_every == 0 or idx == 1:
        print(f"  → {idx}/{len(cpe_list)}   {cpe[:70]}…")

    for vuln in fetch_cves_for_cpe(cpe):
        rows.append(flatten(vuln, cpe))

print("✔️  API queries finished")

# ── 3. build DataFrame & de-dup ────────────────────────────────────────
df = (
    pd.DataFrame(rows)
      .drop_duplicates(subset=["cveID", "cpeName"])
      .reset_index(drop=True)
)
print(f"🗂  {df.shape[0]:,} CVE–CPE rows collected")

📋  1 unique CPEs to query
  → 1/1   cpe:2.3:a:adobe:acrobat_reader:20.004.30006:*:*:*:classic:*:*:*…
✔️  API queries finished
🗂  315 CVE–CPE rows collected


In [14]:
df

cveID                                            cpeName  \
0    CVE-2021-39836  cpe:2.3:a:adobe:acrobat_reader:20.004.30006:*:...   
1    CVE-2021-39837  cpe:2.3:a:adobe:acrobat_reader:20.004.30006:*:...   
2    CVE-2021-39838  cpe:2.3:a:adobe:acrobat_reader:20.004.30006:*:...   
3    CVE-2021-39839  cpe:2.3:a:adobe:acrobat_reader:20.004.30006:*:...   
4    CVE-2021-39840  cpe:2.3:a:adobe:acrobat_reader:20.004.30006:*:...   
..              ...                                                ...   
310  CVE-2025-27161  cpe:2.3:a:adobe:acrobat_reader:20.004.30006:*:...   
311  CVE-2025-27162  cpe:2.3:a:adobe:acrobat_reader:20.004.30006:*:...   
312  CVE-2025-27163  cpe:2.3:a:adobe:acrobat_reader:20.004.30006:*:...   
313  CVE-2025-27164  cpe:2.3:a:adobe:acrobat_reader:20.004.30006:*:...   
314  CVE-2025-27174  cpe:2.3:a:adobe:acrobat_reader:20.004.30006:*:...   

                   published            last_modified     cwes  \
0    2021-09-29T16:15:08.513  2024-11-21T06:20:20.730  CWE-416   
1    2021-09-29T16:15:08.573  2024-11-21T06:20:20.890  CWE-416   
2    2021-09-29T16:15:08.633  2024-11-21T06:20:21.040  CWE-416   
3    2021-09-29T16:15:08.693  2024-11-21T06:20:21.190  CWE-416   
4    2021-09-29T16:15:08.753  2024-11-21T06:20:21.350  CWE-416   
..                       ...                      ...      ...   
310  2025-03-11T18:15:33.993  2025-04-28T16:48:42.960  CWE-125   
311  2025-03-11T18:15:34.150  2025-04-28T16:48:39.727  CWE-824   
312  2025-03-11T18:15:34.293  2025-04-28T16:48:36.780  CWE-125   
313  2025-03-11T18:15:34.437  2025-04-28T16:48:33.017  CWE-125   
314  2025-03-11T18:15:35.600  2025-04-28T16:48:26.390  CWE-416   

                                           description  \
0    Acrobat Reader DC versions 2021.005.20060 (and...   
1    Acrobat Reader DC versions 2021.005.20060 (and...   
2    Acrobat Reader DC versions 2021.005.20060 (and...   
3    Acrobat Reader DC versions 2021.005.20060 (and...   
4    Acrobat Reader DC versions 2021.005.20060 (and...   
..                                                 ...   
310  Acrobat Reader versions 24.001.30225, 20.005.3...   
311  Acrobat Reader versions 24.001.30225, 20.005.3...   
312  Acrobat Reader versions 24.001.30225, 20.005.3...   
313  Acrobat Reader versions 24.001.30225, 20.005.3...   
314  Acrobat Reader versions 24.001.30225, 20.005.3...   

                                            references  \
0    https://helpx.adobe.com/security/products/acro...   
1    https://helpx.adobe.com/security/products/acro...   
2    https://helpx.adobe.com/security/products/acro...   
3    https://helpx.adobe.com/security/products/acro...   
4    https://helpx.adobe.com/security/products/acro...   
..                                                 ...   
310  https://helpx.adobe.com/security/products/acro...   
311  https://helpx.adobe.com/security/products/acro...   
312  https://helpx.adobe.com/security/products/acro...   
313  https://helpx.adobe.com/security/products/acro...   
314  https://helpx.adobe.com/security/products/acro...   

                                                  tags  \
0    Release Notes, Vendor Advisory, Release Notes,...   
1    Release Notes, Vendor Advisory, Release Notes,...   
2    Release Notes, Vendor Advisory, Release Notes,...   
3    Release Notes, Vendor Advisory, Release Notes,...   
4    Release Notes, Vendor Advisory, Release Notes,...   
..                                                 ...   
310                                    Vendor Advisory   
311                                    Vendor Advisory   
312  Vendor Advisory, Technical Description, Third ...   
313  Vendor Advisory, Technical Description, Third ...   
314                                    Vendor Advisory   

                                             full_json  
0    {'cve': {'id': 'CVE-2021-39836', 'sourceIdenti...  
1    {'cve': {'id': 'CVE-2021-39837', 'sourceIdenti...  
2    {'cve': {'id': 'CVE-2021-39838', 'sourceIdenti...  
3    {'cve': 

In [83]:
from pandas import json_normalize

cvssMetricV31 =(
    
    json_normalize(
    df['full_json'],
    record_path=['cve','metrics','cvssMetricV31'],  # drill down two levels
    meta=[ ['cve','id'] ],                         # keep the CVE ID
    )
)

cvssMetricV31.drop(columns=['source','type'],inplace=True,axis=1)
cvssMetricV31

exploitabilityScore  impactScore cvssData.version  \
0                  1.7          2.7              3.1   

                          cvssData.vectorString  cvssData.baseScore  \
0  CVSS:3.1/AV:N/AC:L/PR:H/UI:R/S:C/C:L/I:L/A:N                 4.8   

  cvssData.baseSeverity cvssData.attackVector cvssData.attackComplexity  \
0                MEDIUM               NETWORK                       LOW   

  cvssData.privilegesRequired cvssData.userInteraction cvssData.scope  \
0                        HIGH                 REQUIRED        CHANGED   

  cvssData.confidentialityImpact cvssData.integrityImpact  \
0                            LOW                      LOW   

  cvssData.availabilityImpact          cve.id  
0                        NONE  CVE-2023-26961

In [109]:
weak = (
    json_normalize(
    df['full_json'],
    record_path=['cve','weaknesses'],
    meta=[ ['cve','id'] ],                         # keep the CVE ID
    )
)
weak.drop(columns=['source','description'], inplace=True)
weak

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1 entries, 0 to 0
Data columns (total 4 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   source       1 non-null      object
 1   type         1 non-null      object
 2   description  1 non-null      object
 3   cve.id       1 non-null      object
dtypes: object(4)
memory usage: 164.0+ bytes


type          cve.id
0  Primary  CVE-2023-26961

In [ ]:
weak_desc = (
    json_normalize(
    df['full_json'],
    record_path=['cve','weaknesses', 'description'],
    meta=[ ['cve','id'] ],                         # keep the CVE ID
    )
)

weak_desc.drop(['lang'],inplace=True,axis=1)
weak_desc

value          cve.id
0  CWE-79  CVE-2023-26961

In [82]:
weaknesses = pd.merge(weak_desc, weak, how='inner',on='cve.id')
weaknesses

value          cve.id     type
0  CWE-79  CVE-2023-26961  Primary

In [110]:
descriptions = (
    json_normalize(
    df['full_json'],
    record_path=['cve','descriptions'],
    meta=[ ['cve','id'] ],                         # keep the CVE ID
    )
)

descriptions.drop(columns=['lang'], inplace=True)
descriptions

value          cve.id
0  Alteryx Server 2022.1.1.42590 does not employ ...  CVE-2023-26961

In [111]:
configurations = (
    json_normalize(
    df['full_json'],
    record_path=['cve','configurations'],
    meta=[ ['cve','id'] ],                         # keep the CVE ID
    )
)

#configurations.drop(columns=['lang'], inplace=True)
configurations

nodes          cve.id
0  [{'operator': 'OR', 'negate': False, 'cpeMatch...  CVE-2023-26961

In [113]:
nodes = (
    json_normalize(
    df['full_json'],
    record_path=['cve','configurations', 'nodes'],
    meta=[ ['cve','id'] ],                         # keep the CVE ID
    )
)

#nodes.drop(columns=['lang'], inplace=True)
nodes

operator  negate                                           cpeMatch  \
0       OR   False  [{'vulnerable': True, 'criteria': 'cpe:2.3:a:a...   

           cve.id  
0  CVE-2023-26961

In [117]:
cpeMatch = (
    json_normalize(
    df['full_json'],
    record_path=['cve','configurations','nodes','cpeMatch'],
    meta=[ ['cve','id'] ],                         # keep the CVE ID
    )
)
#cpeMatch.drop(columns=['lang'], inplace=True)
with pd.option_context('display.max_colwidth', None):
    display(cpeMatch['criteria'])

cpeMatch

0    cpe:2.3:a:alteryx:alteryx_server:2022.1.1.42590:*:*:*:*:*:*:*
Name: criteria, dtype: object

vulnerable                                           criteria  \
0        True  cpe:2.3:a:alteryx:alteryx_server:2022.1.1.4259...   

                        matchCriteriaId          cve.id  
0  7E2E8988-A4F4-4C56-9274-0A7F2DA1BD51  CVE-2023-26961

In [92]:
references = (
    json_normalize(
    df['full_json'],
    record_path=['cve', 'references'],
    meta=[ ['cve','id'] ],                         # keep the CVE ID
    )
)


references.info()
#references.drop(columns=['lang'], inplace=True)
references

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4 entries, 0 to 3
Data columns (total 4 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   url     4 non-null      object
 1   source  4 non-null      object
 2   tags    4 non-null      object
 3   cve.id  4 non-null      object
dtypes: object(4)
memory usage: 260.0+ bytes


url  \
0                                 http://alteryx.com   
1  https://gist.github.com/DylanGrl/4269ae834c5d0...   
2                                 http://alteryx.com   
3  https://gist.github.com/DylanGrl/4269ae834c5d0...   

                                 source                             tags  \
0                         cve@mitre.org                [Vendor Advisory]   
1                         cve@mitre.org  [Exploit, Third Party Advisory]   
2  af854a3a-2127-422b-91ae-364da2661108                [Vendor Advisory]   
3  af854a3a-2127-422b-91ae-364da2661108  [Exploit, Third Party Advisory]   

           cve.id  
0  CVE-2023-26961  
1  CVE-2023-26961  
2  CVE-2023-26961  
3  CVE-2023-26961

In [60]:
import pandas as pd
import ast, json

# 0) Inspect what’s actually in your column:
print(df['full_json'].apply(type).value_counts())

# 1) Robust parsing function—if it’s already a dict, leave it alone;
#    if it’s a string, try literal_eval (for Python-style dicts) then fall back
#    to json.loads (for real JSON with double quotes).
def parse_maybe_dict(x):
    if isinstance(x, dict):
        return x
    if isinstance(x, str):
        try:
            return ast.literal_eval(x)
        except (ValueError, SyntaxError):
            return json.loads(x)
    raise TypeError(f"Don't know how to parse {type(x)}")

df['data'] = df['full_json'].apply(parse_maybe_dict)

# 2) Now normalize:
flat = pd.json_normalize(df['data'], sep='_')
flat


full_json
<class 'dict'>    1
Name: count, dtype: int64


cve_id cve_sourceIdentifier            cve_published  \
0  CVE-2023-26961        cve@mitre.org  2023-08-08T20:15:10.080   

          cve_lastModified cve_vulnStatus cve_cveTags  \
0  2024-11-21T07:52:07.460       Modified          []   

                                    cve_descriptions  \
0  [{'lang': 'en', 'value': 'Alteryx Server 2022....   

                           cve_metrics_cvssMetricV31  \
0  [{'source': 'nvd@nist.gov', 'type': 'Primary',...   

                                      cve_weaknesses  \
0  [{'source': 'nvd@nist.gov', 'type': 'Primary',...   

                                  cve_configurations  \
0  [{'nodes': [{'operator': 'OR', 'negate': False...   

                                      cve_references  
0  [{'url': 'http://alteryx.com', 'source': 'cve@...

In [101]:
with pd.option_context('display.max_colwidth', None):
    display(df['full_json'])

0    {'cve': {'id': 'CVE-2023-26961', 'sourceIdentifier': 'cve@mitre.org', 'published': '2023-08-08T20:15:10.080', 'lastModified': '2024-11-21T07:52:07.460', 'vulnStatus': 'Modified', 'cveTags': [], 'descriptions': [{'lang': 'en', 'value': 'Alteryx Server 2022.1.1.42590 does not employ file type verification for uploaded files. This vulnerability allows attackers to upload arbitrary files (e.g., JavaScript content for stored XSS) via the type field in a JSON document within a PUT /gallery/api/media request.'}], 'metrics': {'cvssMetricV31': [{'source': 'nvd@nist.gov', 'type': 'Primary', 'cvssData': {'version': '3.1', 'vectorString': 'CVSS:3.1/AV:N/AC:L/PR:H/UI:R/S:C/C:L/I:L/A:N', 'baseScore': 4.8, 'baseSeverity': 'MEDIUM', 'attackVector': 'NETWORK', 'attackComplexity': 'LOW', 'privilegesRequired': 'HIGH', 'userInteraction': 'REQUIRED', 'scope': 'CHANGED', 'confidentialityImpact': 'LOW', 'integrityImpact': 'LOW', 'availabilityImpact': 'NONE'}, 'exploitabilityScore': 1.7, 'impactScore': 2